In [15]:
# Import dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd

In [16]:
# Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path)

## Article Scraping

In [3]:
# Visit the mars nasa news site
url = 'http://mars.nasa.gov/news/'
browser.visit(url)

# Optional delay for loading the page
# for pages that take a while to load, especially if image-heavy
# tells the browser to wait a second before searching for components
browser.is_element_present_by_css('ul.item_list li.slide', wait_time = 1)

True

In [4]:
# Set up HTML parser
html = browser.html
news_soup = BeautifulSoup(html, 'html.parser')

# Parent element that holds all other elements within it
# Reference it to filter search results even further
# ul tag with class 'item_list', li tag with class 'slide'
slide_elem = news_soup.select_one('ul.item_list li.slide')

In [5]:
# Begin scraping
slide_elem.find('div', class_ = 'content_title')

slide_elem

<li class="slide"><div class="image_and_description_container"><a href="/news/8654/how-nasas-perseverance-mars-team-adjusted-to-work-in-the-time-of-coronavirus/" target="_self"><div class="rollover_description"><div class="rollover_description_inner">Like much of the rest of the world, the Mars rover team is pushing forward with its mission-critical work while putting the health and safety of their colleagues and community first.</div><div class="overlay_arrow"><img alt="More" src="/assets/overlay-arrow.png"/></div></div><div class="list_image"><img alt="Members of NASA's Perseverance rover mission work remotely from home during the coronavirus outbreak." src="/system/news_items/list_view_images/8654_MAIN-IMAGE-PIA23881-320x240.jpg"/></div><div class="bottom_gradient"><div><h3>How NASA's Perseverance Mars Team Adjusted to Work in the Time of Coronavirus </h3></div></div></a><div class="list_text"><div class="list_date">April 21, 2020</div><div class="content_title"><a href="/news/8654/

In [6]:
# Use the parent element to find the first 'a' tag and save it as news_title
news_title = slide_elem.find('div', class_ = 'content_title').get_text()

news_title

"How NASA's Perseverance Mars Team Adjusted to Work in the Time of Coronavirus "

In [7]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_ = 'article_teaser_body').get_text()

news_p

'Like much of the rest of the world, the Mars rover team is pushing forward with its mission-critical work while putting the health and safety of their colleagues and community first.'

## Image Scraping

### Featured Images

In [69]:
# Visit URL
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

browser.visit(url)

In [42]:
# Find and click the full image button
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

In [43]:
# Find the more info button and click that
browser.is_element_present_by_text('more info', wait_time = 1)
more_info_elem = browser.find_link_by_partial_text('more info')
more_info_elem.click()

C:\Users\Helen\anaconda3\envs\PythonData\lib\site-packages\splinter\driver\webdriver\__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [44]:
# Parse the resulting html with soup
html = browser.html
img_soup = BeautifulSoup(html, 'html.parser')

In [12]:
# Find the relative image url
img_url_rel = img_soup.select_one('figure.lede a img').get('src')
img_url_rel

'/spaceimages/images/largesize/PIA17302_hires.jpg'

In [13]:
# Use the base URL to create an absolute URL
img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17302_hires.jpg'

### Mars Hemisphere Scraping

In [17]:
# Visit URL
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(url)

In [18]:
# Parse the resulting html with soup
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

base_url= 'https://astrogeology.usgs.gov'

#Parse out the sourp and find all of the links
description_elem = soup.find_all('div', class_='description')

# for each item in desc_elem, pull the link and concatenate with base_url
links = [base_url + item.a['href'] for item in description_elem]

links

['https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']

In [30]:
# Initialize list for dict
title_url_list = []

# Parse through each link to grab title and img_url
for link in links:
    # visit each link
    browser.visit(link)
    
    # Parse the resulting html with soup
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    # Get title
    title = soup.select_one('h2.title').get_text()
    
    # Get img_url
    img_elem = soup.select('div.downloads li')
    
    # first_element, = [OUTPUT FOR_LOOP FILTER]   
    img_url, = [item.a['href'] for item in img_elem if item.a.get_text() == 'Sample']
    
    
    # Put into a dictionary
    title_url_dict = {'title': title, 'img_url': img_url}
    
    # Append to list
    title_url_list.append(title_url_dict)
    
print(title_url_list)


['http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg', 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg', 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg', 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg']


## Facts Scraping

In [14]:
# Scrape the entire table with Pandas
df = pd.read_html('http://space-facts.com/mars/')[0] # read html table into DF, [0] = read the first table it encounters
df.columns = ['description', 'value'] # columns for the new DF
df. set_index('description', inplace = True)
df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [15]:
# Convert DF back into html
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

In [16]:
# End automated browsing session
browser.quit()